In [1]:
import tensorflow as tf
import DataLoader
from tensorflow.keras import layers
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
from config import Config
import matplotlib.pyplot as plt

In [2]:
train_dir = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/train"
val_dir = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/val"
test_dir = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/test"
csv_path = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/df.csv"

dataset_train = DataLoader.Dataset()
#dataset_test = DataLoader.Dataset()
#dataset_val = DataLoader.Dataset()

dataset_train.load_dataset(train_dir)
#dataset_test.load_dataset(test_dir)
#dataset_val.load_dataset(val_dir)

/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/train


In [3]:
case_ids_sort = sorted(dataset_train.case_id)
train_id = case_ids_sort[:66]
val_id = case_ids_sort[66:]
print(train_id, val_id)

['10_control', '10_event', '11_control', '11_event', '12_control', '12_event', '13_control', '13_event', '14_control', '14_event', '15_control', '15_event', '16_control', '16_event', '17_control', '17_event', '18_control', '18_event', '19_control', '19_event', '1_control', '1_event', '20_control', '20_event', '21_control', '21_event', '22_control', '23_control', '23_event', '24_control', '24_event', '25_control', '25_event', '26_control', '26_event', '27_control', '27_event', '28_control', '28_event', '29_control', '29_event', '2_control', '2_event', '30_control', '30_event', '31_control', '31_event', '32_control', '32_event', '33_control', '33_event', '34_control', '34_event', '35_control', '35_event', '36_control', '36_event', '37_control', '37_event', '38_control', '38_event', '39_control', '39_event', '3_control', '3_event', '40_control'] ['40_event', '41_control', '41_event', '4_control', '4_event', '5_control', '5_event', '6_control', '6_event', '7_control', '7_event', '8_control

In [4]:
def handle_case_attributes(csv_path):
    df = pd.read_csv(csv_path,header=0)
    df.drop(['score2', "tchol",'hdl','ldl','trig','hypotension'], axis=1, inplace=True)
    df['ane_diam'].fillna(0,inplace=True)
    df.fillna(method='ffill',inplace=True)
    small = LabelBinarizer().fit(df["smallbin"])
    df['smallbin']=small.transform(df['smallbin'])
    dia = LabelBinarizer().fit(df["diabetes"])
    hbp = LabelBinarizer().fit(df["HBP"])
    df['diabetes']=dia.transform(df['diabetes'])
    df['HBP']=hbp.transform(df['HBP'])
    #cs = MinMaxScaler()
    #df[df.columns[2:]] = cs.fit_transform(df[df.columns[2:]])
    #df[df.columns[2:]] = df[df.columns[2:]].astype(np.float32)
    return df

In [5]:
def train_generator():
    for i in train_id:
        img, class_id = dataset_train.load_case_image(i, IMG_ID)
        det = augment.to_deterministic()
        img = det.augment_image(img)
        yield img, class_id

def test_generator():
    for i in val_id:
        img, class_id = dataset_test.load_case_image(i, IMG_ID)[0]
        yield img

In [6]:
df = handle_case_attributes(csv_path)
print(df)

    id  group  sex  age  smallbin        bmi  time_onset  ane_diam  diabetes  \
0    1      1    0   71         1  22.656250         7.0     5.000         1   
1    1      0    0   69         1  27.548209        48.0     5.640         1   
2    2      1    1   33         0  25.380000        12.0     4.700         0   
3    2      0    1   28         0  29.069767        10.0     0.000         0   
4    3      0    1   50         0  23.875115       168.0     0.000         0   
5    3      1    1   49         1  27.379665        24.0     5.100         0   
6    4      0    1   47         0  40.000000         5.0     0.000         0   
7    4      1    1   53         1  19.390582        24.0     4.000         0   
8    5      1    1   50         0  31.141869        24.0     4.980         0   
9    5      0    1   50         0  26.423570         8.0     5.600         0   
10   6      0    1   46         0  29.410759         4.0     4.400         0   
11   6      1    1   50         0  24.22

In [7]:
pre_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', 
                                        input_shape=(Config.IMAGE_DIM,Config.IMAGE_DIM,3))
x = pre_model.output
x = layers.GlobalAveragePooling2D()(x)
model = tf.keras.Model(inputs=pre_model.input, outputs=x)
for i in range(14):
    IMG_ID = i+1
    df[str(IMG_ID)]=0
    for j in dataset_train.case_id:
        img = dataset_train.load_case_image(j,IMG_ID)[0]
        #print(img.shape)
        img = np.expand_dims(img,axis=0)
        features = model.predict(img)
        
        features = np.squeeze(features)
        f = np.mean(features)
        #f = np.dot(features,features.T)
        print(f)
        
        cid, group = j.split("_")
        if group=='event' :
            #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==1)] = f
            df.loc[(df['id']==int(cid))&(df['group']==1), str(IMG_ID)] = f
        else:
            df.loc[(df['id']==int(cid))&(df['group']==0), str(IMG_ID)] = f
            #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==0)] = f
        
        #print(df)
    
    # for j in val_id:
    #     img = dataset_val.load_case_image(j,IMG_ID)[0]
    #     #print(img.shape)
    #     img = np.expand_dims(img,axis=0)
    #     features = model.predict(img)
    #     features = np.squeeze(features)
    #     #f = np.mean(features)
    #     f = np.dot(features,features.T)
    #     print(f)
    #     cid, group = j.split("_")
    #     if group=='event' :
    #         #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==1)] = f
    #         df.loc[(df['id']==int(cid))&(df['group']==1), str(IMG_ID)] = f
    #     else:
    #         df.loc[(df['id']==int(cid))&(df['group']==0), str(IMG_ID)] = f
    #         #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==0)] = f
        #print(df)
print(df)
  
    
    
    
    
    

0.70017505
0.72640586
0.771801
0.72780186
0.6094866
0.81275123
0.8422905
0.73188365
0.8249871
0.7358544
0.7930662
0.5930481
0.62000024
0.5804287
0.60801256
0.6534
0.6183977
0.60659164
0.75420576
0.6224841
0.6880071
0.60377955
0.7006806
0.57800084
0.52356124
0.80328625
0.5924973
0.76819146
0.6576755
0.7578819
0.6741388
0.6594148
0.6672804
0.6586585
0.74061584
0.67228293
0.75028354
0.69165564
0.73694026
0.5408187
0.7209084
0.6475097
0.7081269
0.5971956
0.7536216
0.60351694
0.7527619
0.822083
0.65236604
0.64451236
0.6117256
0.8193367
0.89595526
0.7410426
0.9433916
0.62544286
0.5956259
0.7706926
0.82871675
0.6776608
0.83374846
1.1696804
0.691242
0.80181444
0.72976416
1.0107079
0.75138795
0.5407445
0.76903266
0.5942495
0.648216
0.69498765
0.6833655
0.76697284
0.7481431
0.9376172
0.7000346
0.6045645
0.6136874
0.78653824
0.73682886
0.67471576
0.7666874
0.74830693
0.8580129
0.66959375
1.0671157
0.9336258
0.6945363
1.0503814
0.62827945
1.1285763
0.6660514
0.7357551
1.1982613
0.7157098
0.7046780

0.68531835
0.6230217
0.8367733
1.0715703
0.828861
1.0370865
0.85924363
0.6585183
0.761068
0.5857002
0.6743121
0.56495637
0.8799412
0.6074356
1.0203578
0.9524759
0.9984677
0.98876125
0.52976304
0.94452846
0.54596555
0.5809745
0.88779056
0.9071754
0.75945354
0.65654314
0.80073833
1.0145342
0.74693
0.57596517
0.8426225
0.7578953
0.67425394
0.8656935
0.91253316
0.61109495
0.72668815
0.65226007
0.7492826
0.6248001
0.6139937
0.768508
0.73343587
0.9173385
0.78289413
1.0572194
0.6599628
0.83579725
0.74634695
0.6304122
0.8027597
0.84094274
0.805909
0.6881845
0.49406278
0.8187823
0.72241074
0.7142036
0.6285855
0.9415298
0.91105604
1.0689306
1.0721866
0.86294043
0.6726197
0.5449244
0.96036375
0.6857406
0.5572175
0.81792295
0.7857338
0.7210391
0.7695488
0.7339059
0.703288
0.6217461
0.84124696
0.9867611
0.6405581
0.71142066
0.84348434
0.5835073
0.89060783
0.72418493
0.9813667
0.7238681
0.9564561
0.7056757
0.7908016
0.7638436
0.59294504
0.6682799
0.6775679
0.8660101
0.67651176
1.0979711
0.95288694
0

In [8]:
df

,id,group,sex,age,smallbin,bmi,time_onset,ane_diam,diabetes,hs_cad,...,5,6,7,8,9,10,11,12,13,14
0,1,1,0,71,1,22.656250,7.0,5.000,1,0.0,...,0.848796,0.812114,0.716055,0.691321,0.644493,0.668649,0.624800,0.699392,0.786897,0.823512
1,1,0,0,69,1,27.548209,48.0,5.640,1,0.0,...,0.650145,0.570830,0.572118,0.512155,0.487567,0.490104,0.613994,0.711345,0.716287,0.753134
2,2,1,1,33,0,25.380000,12.0,4.700,0,0.0,...,1.251096,1.191099,1.116717,1.132562,1.053531,1.018457,0.843484,0.731798,0.879423,0.890308
3,2,0,1,28,0,29.069767,10.0,0.000,0,0.0,...,0.722984,0.821816,0.687388,0.741141,0.684289,0.677083,0.711421,0.717909,0.812973,0.897118
4,3,0,1,50,0,23.875115,168.0,0.000,0,0.0,...,0.822045,0.715562,0.730932,0.638460,0.890284,0.986109,0.733906,0.944759,0.905293,0.921894
5,3,1,1,49,1,27.379665,24.0,5.100,0,0.0,...,0.968342,0.979928,0.837269,0.855850,0.783220,0.798221,0.703288,0.785893,0.770986,0.807373
6,4,0,1,47,0,40.000000,5.0,0.000,0,0.0,...,0.857692,1.006559,0.933881,0.889987,0.826659,0.866139,0.817923,0.828984,0.907569,0.949058
7,4,1,1,53,1,19.390582,24.0,4.000,0,0.0,...,1.214223,1.274913,1.116541,0.849687,0.993176,1.125274,0.785734,0.807925,0.703367,0.951241
8,5,1,1,50,0,31.141869,24.0,4.980,0,0.0,...,1.265966,1.112444,1.061088,1.047844,1.067729,1.071570,0.981367,0.968657,1.006463,1.047577
9,5,0,1,50,0,26.423570,8.0,5.600,0,0.0,...,0.704917,0.643494,0.817154,0.864888,0.865247,0.836773,0.724185,0.695407,0.771049,0.809159


In [9]:
df.to_csv('/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/df_new.csv', index=False)

In [10]:
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [11]:
cs = MinMaxScaler()
df[df.columns[2:]] = cs.fit_transform(df[df.columns[2:]])
df[df.columns[2:]] = df[df.columns[2:]].astype(np.float32)

In [13]:
X = df[df.columns[2:]]
Y = df["group"]

In [16]:
train_l = []
for i in train_id:
    cid, group = i.split("_")
    if group=='event' :
        #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==1)] = f
        df1 = df.loc[(df['id']==int(cid))&(df['group']==1)]
    else:
        df1 = df.loc[(df['id']==int(cid))&(df['group']==0)]
    print(df1)
    train_l.append(df1)
train_df = pd.concat(train_l,axis=0)
print(train_df)

    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
19  10      0  1.0  0.638298       0.0  0.468085         0.0  0.586667   

    diabetes  hs_cad  ...         5         6         7         8         9  \
19       0.0     0.0  ...  0.740703  0.656332  0.443837  0.604565  0.656029   

          10        11        12        13       14  
19  0.674052  0.684056  0.650831  0.702614  0.69505  

[1 rows x 33 columns]
    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
18  10      1  1.0  0.744681       0.0  0.297872         0.0  0.617333   

    diabetes  hs_cad  ...         5         6         7         8         9  \
18       0.0     0.0  ...  0.818681  0.722436  0.802851  0.768232  0.676302   

         10        11        12        13        14  
18  0.77932  0.716767  0.522419  0.522961  0.603904  

[1 rows x 33 columns]
    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
20  11      0  1.0  0.319149       1.0  0.510638

[1 rows x 33 columns]
    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
59  30      1  1.0  0.510638       0.0  0.302314    0.021084       0.6   

    diabetes  hs_cad  ...         5         6         7         8         9  \
59       0.0     0.0  ...  0.653395  0.577668  0.606761  0.641247  0.550155   

          10        11        12        13        14  
59  0.541996  0.666119  0.580676  0.692156  0.671322  

[1 rows x 33 columns]
    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
61  31      0  1.0  0.425532       0.0  0.613181    0.060241     0.684   

    diabetes  hs_cad  ...         5         6         7         8         9  \
61       0.0     0.0  ...  0.656932  0.503537  0.570845  0.550328  0.503132   

          10        11        12        13        14  
61  0.658891  0.763657  0.749464  0.781238  0.769005  

[1 rows x 33 columns]
    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
60  31      1  1.0  0.

In [17]:
val_l = []
for i in val_id:
    cid, group = i.split("_")
    if group=='event' :
        #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==1)] = f
        df1 = df.loc[(df['id']==int(cid))&(df['group']==1)]
    else:
        df1 = df.loc[(df['id']==int(cid))&(df['group']==0)]
    print(df1)
    val_l.append(df1)
val_df = pd.concat(val_l,axis=0)
print(val_df)

    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
78  40      1  0.0  0.574468       1.0  0.142233    0.060241       0.0   

    diabetes  hs_cad  ...        5         6         7         8         9  \
78       0.0     0.0  ...  0.52581  0.589495  0.534044  0.636523  0.558828   

          10        11        12        13        14  
78  0.593304  0.597432  0.585729  0.585674  0.644715  

[1 rows x 33 columns]
    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
81  41      0  0.0  0.765957       1.0  0.450216     0.13253  0.586667   

    diabetes  hs_cad  ...         5         6         7         8        9  \
81       1.0     0.0  ...  0.526547  0.461764  0.330176  0.779233  0.79941   

          10        11        12        13        14  
81  0.794614  0.804842  0.799231  0.992835  0.808628  

[1 rows x 33 columns]
    id  group  sex       age  smallbin       bmi  time_onset  ane_diam  \
80  41      1  0.0  0.638298       1.0  0.437621

In [18]:
train_X = train_df[train_df.columns[2:]]
train_Y = train_df["group"]
val_X = val_df[val_df.columns[2:]]
val_Y = val_df["group"]

In [21]:

from sklearn.feature_selection import f_classif
 
from sklearn.feature_selection import SelectKBest
 
# 代表选择特征值的数量
 
selectkBest=SelectKBest(
        f_classif,
        k=5
        )
 
# 选择自变量
 
feature=train_X
 
# 调用fit方法进行最好特征的选择
 
bestFeature=selectkBest.fit_transform(
        feature,
        train_Y
        )
 
# 查看关键因子
print(selectkBest.get_support())
feature.columns[selectkBest.get_support()]
 

[False False False False  True False False False False False False False
 False False False False False False False False False  True False False
 False False False  True  True  True False]


Index(['time_onset', '5', '11', '12', '13'], dtype='object')

from sklearn.feature_selection import VarianceThreshold

#方差选择法，返回值为特征选择后的数据
#参数threshold为方差的阈值
VarianceThreshold(threshold=).fit_transform(X)

In [26]:
for k in range(31):
    selectkBest=SelectKBest(f_classif,k=k+1)
 
    # 选择自变量
 
    feature=X
 
    # 调用fit方法进行最好特征的选择
 
    bestFeature=selectkBest.fit_transform(feature, Y)
 
    # 查看关键因子
    #print(selectkBest.get_support())
    #print(feature.columns[selectkBest.get_support()])
    #for i in range(30):
    x = X[feature.columns[selectkBest.get_support()]]
    metric_all = pd.DataFrame()

    from sklearn.linear_model import LogisticRegression
 
    lr = LogisticRegression(C=1000, solver='liblinear', random_state=0)
    metric = cross_val_score(lr, X, Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['glm'] = metric[::-1]
 
    # 拟合决策树模型
    from sklearn import tree
 
    tree = tree.DecisionTreeClassifier(criterion='gini')
    metric = cross_val_score(tree, X, Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['tree'] = metric[::-1]

    # 拟合svm模型
    from sklearn import svm

    svc = svm.SVC(C=1.0, kernel='rbf', gamma='auto')
    metric = cross_val_score(svc, X, Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['svm'] = metric[::-1]

    # 拟合随机森林算法
    from sklearn.ensemble import RandomForestClassifier

    RF = RandomForestClassifier(n_estimators=30, criterion='gini', random_state=10)
    metric = cross_val_score(RF, X, Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['RandomForest'] = metric[::-1]

    # 构造knn最近邻模型
    from sklearn import neighbors

    knn = neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
    metric = cross_val_score(estimator=knn, X=X, y=Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['knn'] = metric[::-1]

    # 构造lda模型
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

    lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None, priors=None)
    metric = cross_val_score(estimator=lda, X=X, y=Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['lda'] = metric[::-1]

    # 构造qda模型
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

    qda = QuadraticDiscriminantAnalysis()
    metric = cross_val_score(estimator=qda, X=X, y=Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['qda'] = metric[::-1]

    #  高斯朴素贝叶斯,多项式贝叶斯适用于文本分类，伯努利贝叶斯需要全部变量威二值变量
    from sklearn.naive_bayes import GaussianNB

    gnb = GaussianNB()
    metric = cross_val_score(estimator=gnb, X=X, y=Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['gnb'] = metric[::-1]

    # sklearn提供了bp多层神经网络，隐含层设为 (4, 3, 2)
    from sklearn.neural_network import MLPClassifier

    mbp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 3, 2), random_state=1)
    metric = cross_val_score(estimator=mbp, X=X, y=Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['mbp'] = metric[::-1]

    # 采用adaboost算法
    from sklearn.ensemble import AdaBoostClassifier

    # 弱分类器的参数 base_estimator 默认为决策树
    ada = AdaBoostClassifier(n_estimators=100)
    metric = cross_val_score(estimator=ada, X=X, y=Y, cv=10, scoring='roc_auc')
    metric.sort()
    metric_all['ada'] = metric[::-1]

    # 将10种模型结果进行比较
    print(k+1,"个特征")
    print(metric_all)
    print("按照accuracy进行排序")
    metric_mean = metric_all.mean()
    print(metric_mean.sort_values(ascending=False))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

1 个特征
      glm   tree     svm  RandomForest      knn     lda     qda      gnb  mbp  \
0  0.7500  1.000  0.8750       0.90625  0.96875  0.7500  0.7500  1.00000  0.5   
1  0.7500  0.750  0.8750       0.71875  0.87500  0.7500  0.5000  0.81250  0.5   
2  0.7500  0.750  0.8125       0.68750  0.71875  0.7500  0.5000  0.78125  0.5   
3  0.6875  0.750  0.6875       0.68750  0.71875  0.7200  0.5000  0.75000  0.5   
4  0.6250  0.750  0.6250       0.68750  0.65625  0.6250  0.3750  0.75000  0.5   
5  0.6250  0.625  0.6250       0.65625  0.62500  0.6250  0.3125  0.68750  0.5   
6  0.6250  0.625  0.5000       0.59375  0.59375  0.6250  0.2500  0.62500  0.5   
7  0.5600  0.500  0.3750       0.46875  0.53125  0.5625  0.2500  0.62500  0.5   
8  0.5000  0.500  0.3750       0.46000  0.40625  0.5000  0.2500  0.50000  0.5   
9  0.5000  0.500  0.2000       0.34375  0.18000  0.5000  0.1875  0.31250  0.5   

      ada  
0  0.7500  
1  0.6250  
2  0.6000  
3  0.5625  
4  0.5625  
5  0.5625  
6  0.5000  
7  0.4

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

2 个特征
      glm   tree     svm  RandomForest      knn     lda     qda      gnb  mbp  \
0  0.7500  1.000  0.8750       0.90625  0.96875  0.7500  0.7500  1.00000  0.5   
1  0.7500  0.875  0.8750       0.71875  0.87500  0.7500  0.5000  0.81250  0.5   
2  0.7500  0.750  0.8125       0.68750  0.71875  0.7500  0.5000  0.78125  0.5   
3  0.6875  0.750  0.6875       0.68750  0.71875  0.7200  0.5000  0.75000  0.5   
4  0.6250  0.750  0.6250       0.68750  0.65625  0.6250  0.3750  0.75000  0.5   
5  0.6250  0.625  0.6250       0.65625  0.62500  0.6250  0.3125  0.68750  0.5   
6  0.6250  0.500  0.5000       0.59375  0.59375  0.6250  0.2500  0.62500  0.5   
7  0.5600  0.500  0.3750       0.46875  0.53125  0.5625  0.2500  0.62500  0.5   
8  0.5000  0.500  0.3750       0.46000  0.40625  0.5000  0.2500  0.50000  0.5   
9  0.5000  0.500  0.2000       0.34375  0.18000  0.5000  0.1875  0.31250  0.5   

      ada  
0  0.7500  
1  0.6250  
2  0.6000  
3  0.5625  
4  0.5625  
5  0.5625  
6  0.5000  
7  0.4

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

...

29 个特征
      glm   tree     svm  RandomForest      knn     lda     qda      gnb  mbp  \
0  0.7500  1.000  0.8750       0.90625  0.96875  0.7500  0.7500  1.00000  0.5   
1  0.7500  0.750  0.8750       0.71875  0.87500  0.7500  0.5000  0.81250  0.5   
2  0.7500  0.750  0.8125       0.68750  0.71875  0.7500  0.5000  0.78125  0.5   
3  0.6875  0.750  0.6875       0.68750  0.71875  0.7200  0.5000  0.75000  0.5   
4  0.6250  0.750  0.6250       0.68750  0.65625  0.6250  0.3750  0.75000  0.5   
5  0.6250  0.625  0.6250       0.65625  0.62500  0.6250  0.3125  0.68750  0.5   
6  0.6250  0.625  0.5000       0.59375  0.59375  0.6250  0.2500  0.62500  0.5   
7  0.5600  0.625  0.3750       0.46875  0.53125  0.5625  0.2500  0.62500  0.5   
8  0.5000  0.500  0.3750       0.46000  0.40625  0.5000  0.2500  0.50000  0.5   
9  0.5000  0.500  0.2000       0.34375  0.18000  0.5000  0.1875  0.31250  0.5   

      ada  
0  0.7500  
1  0.6250  
2  0.6000  
3  0.5625  
4  0.5625  
5  0.5625  
6  0.5000  
7  0.

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

30 个特征
      glm   tree     svm  RandomForest      knn     lda     qda      gnb  mbp  \
0  0.7500  1.000  0.8750       0.90625  0.96875  0.7500  0.7500  1.00000  0.5   
1  0.7500  0.750  0.8750       0.71875  0.87500  0.7500  0.5000  0.81250  0.5   
2  0.7500  0.750  0.8125       0.68750  0.71875  0.7500  0.5000  0.78125  0.5   
3  0.6875  0.750  0.6875       0.68750  0.71875  0.7200  0.5000  0.75000  0.5   
4  0.6250  0.625  0.6250       0.68750  0.65625  0.6250  0.3750  0.75000  0.5   
5  0.6250  0.625  0.6250       0.65625  0.62500  0.6250  0.3125  0.68750  0.5   
6  0.6250  0.625  0.5000       0.59375  0.59375  0.6250  0.2500  0.62500  0.5   
7  0.5600  0.500  0.3750       0.46875  0.53125  0.5625  0.2500  0.62500  0.5   
8  0.5000  0.500  0.3750       0.46000  0.40625  0.5000  0.2500  0.50000  0.5   
9  0.5000  0.500  0.2000       0.34375  0.18000  0.5000  0.1875  0.31250  0.5   

      ada  
0  0.7500  
1  0.6250  
2  0.6000  
3  0.5625  
4  0.5625  
5  0.5625  
6  0.5000  
7  0.

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

31 个特征
      glm   tree     svm  RandomForest      knn     lda     qda      gnb  mbp  \
0  0.7500  0.875  0.8750       0.90625  0.96875  0.7500  0.7500  1.00000  0.5   
1  0.7500  0.750  0.8750       0.71875  0.87500  0.7500  0.5000  0.81250  0.5   
2  0.7500  0.750  0.8125       0.68750  0.71875  0.7500  0.5000  0.78125  0.5   
3  0.6875  0.750  0.6875       0.68750  0.71875  0.7200  0.5000  0.75000  0.5   
4  0.6250  0.625  0.6250       0.68750  0.65625  0.6250  0.3750  0.75000  0.5   
5  0.6250  0.625  0.6250       0.65625  0.62500  0.6250  0.3125  0.68750  0.5   
6  0.6250  0.625  0.5000       0.59375  0.59375  0.6250  0.2500  0.62500  0.5   
7  0.5600  0.500  0.3750       0.46875  0.53125  0.5625  0.2500  0.62500  0.5   
8  0.5000  0.500  0.3750       0.46000  0.40625  0.5000  0.2500  0.50000  0.5   
9  0.5000  0.500  0.2000       0.34375  0.18000  0.5000  0.1875  0.31250  0.5   

      ada  
0  0.7500  
1  0.6250  
2  0.6000  
3  0.5625  
4  0.5625  
5  0.5625  
6  0.5000  
7  0.

In [19]:
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score
def calculate_metric(gt, pred): 
    confusion = confusion_matrix(gt,pred)
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    acc = (TP+TN)/float(TP+TN+FP+FN)
    sen = TP / float(TP+FN)
    spe = TN / float(TN+FP)
    return acc, sen, spe
    #print('Accuracy:',acc)
    #print('Sensitivity:',sen)
    #print('Specificity:',spe) 

In [32]:
pre = [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0]
auc_lr = roc_auc_score(val_Y, pre)
print(calculate_metric(val_Y, pre))
print(auc_lr)

(0.2, 0.125, 0.2857142857142857)
0.20535714285714285


In [25]:
for k in range(31):
    selectkBest=SelectKBest(f_classif,k=k+1)
 
    # 选择自变量
 
    feature=train_X
 
    # 调用fit方法进行最好特征的选择
 
    bestFeature=selectkBest.fit_transform(feature, train_Y)
 
    # 查看关键因子
    #print(selectkBest.get_support())
    #print(feature.columns[selectkBest.get_support()])
    #for i in range(30):
    x = train_X[feature.columns[selectkBest.get_support()]]
    metric_all = pd.DataFrame()

    from sklearn.linear_model import LogisticRegression
 
    lr = LogisticRegression(C=1000, solver='liblinear', random_state=0)
    pre_lr = lr.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_lr = roc_auc_score(val_Y, pre_lr)
    if auc_lr>0.7:
        print('AUC_lr:', auc_lr)
        print(calculate_metric(val_Y, pre_lr))
    #metric.sort()
    metric_all['glm'] = auc_lr
 
    # 拟合决策树模型
    from sklearn import tree
 
    tree = tree.DecisionTreeClassifier(criterion='gini')
    pre_tree = tree.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_tree = roc_auc_score(val_Y, pre_tree)
    if auc_tree>0.7:
        print('AUC_tree:', auc_tree)
        print(calculate_metric(val_Y, pre_tree))
    #metric = cross_val_score(tree, x, Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['tree'] = auc_tree
 
    # 拟合svm模型
    from sklearn import svm
 
    svc = svm.SVC(C=1.0, kernel='rbf', gamma='auto')
    pre_svc = svc.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_svc = roc_auc_score(val_Y, pre_svc)
    if auc_svc>0.7:
        print('AUC_svc:', auc_svc)
        print(calculate_metric(val_Y, pre_svc))
    #metric = cross_val_score(svc, x, Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['svm'] = auc_svc
 
    # 拟合随机森林算法
    from sklearn.ensemble import RandomForestClassifier
 
    RF = RandomForestClassifier(n_estimators=30, criterion='gini', random_state=10)
    pre_RF = RF.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_RF = roc_auc_score(val_Y, pre_RF)
    if auc_RF>0.7:
        print('AUC_RF:', auc_RF)
        print(calculate_metric(val_Y, pre_RF))
    #metric = cross_val_score(RF, x, Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['RandomForest'] = auc_RF
 
    # 构造knn最近邻模型
    from sklearn import neighbors
 
    knn = neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
    pre_knn = knn.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_knn = roc_auc_score(val_Y, pre_knn)
    if auc_knn>0.7:
        print('AUC_knn:', auc_knn)
        print(calculate_metric(val_Y, pre_knn))
    #metric = cross_val_score(estimator=knn, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['knn'] = auc_knn
 
    # 构造lda模型
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 
    lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None, priors=None)
    pre_lda = lda.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_lda = roc_auc_score(val_Y, pre_lda)
    if auc_lda>0.7:
        print('AUC_lda:', auc_lda)
        print(calculate_metric(val_Y, pre_lda))
    #metric = cross_val_score(estimator=lda, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['lda'] = auc_lda
 
    # 构造qda模型
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
 
    qda = QuadraticDiscriminantAnalysis()
    pre_qda = qda.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_qda = roc_auc_score(val_Y, pre_qda)
    if auc_qda>0.7:
        print('AUC_qda:', auc_qda)
        print(calculate_metric(val_Y, pre_qda))
    #metric = cross_val_score(estimator=qda, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['qda'] = auc_qda
 
    #  高斯朴素贝叶斯,多项式贝叶斯适用于文本分类，伯努利贝叶斯需要全部变量威二值变量
    from sklearn.naive_bayes import GaussianNB
 
    gnb = GaussianNB()
    pre_gnb = gnb.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_gnb = roc_auc_score(val_Y, pre_gnb)
    if auc_gnb>0.7:
        print('AUC_gnb:', auc_gnb)
        print(calculate_metric(val_Y, pre_gnb))
    #metric = cross_val_score(estimator=gnb, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['gnb'] = auc_gnb
 
    # sklearn提供了bp多层神经网络，隐含层设为 (4, 3, 2)
    from sklearn.neural_network import MLPClassifier
 
    mbp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 3, 2), random_state=1)
    pre_mbp = mbp.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_mbp = roc_auc_score(val_Y, pre_mbp)
    if auc_mbp>0.7:
        print('AUC_mbp:', auc_mbp)
        print(calculate_metric(val_Y, pre_mbp))
    #metric = cross_val_score(estimator=mbp, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['mbp'] = auc_mbp
 
    # 采用adaboost算法
    from sklearn.ensemble import AdaBoostClassifier
 
    # 弱分类器的参数 base_estimator 默认为决策树
    ada = AdaBoostClassifier(n_estimators=100)
    pre_ada = ada.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
    auc_ada = roc_auc_score(val_Y, pre_ada)
    if auc_ada>0.7:
        print('AUC_ada:', auc_ada)
        print(calculate_metric(val_Y, pre_ada))
    #metric = cross_val_score(estimator=ada, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    metric_all['ada'] = auc_ada
 
    # 将10种模型结果进行比较
    print(metric_all)
    print(k+1,"..............................................................")#,"个特征按照accuracy进行排序")
    #metric_mean = metric_all.mean()
    #print(metric_mean.sort_values(ascending=False))
    #score = cross_val_score(g, X[feature.columns[selectkBest.get_support()]], Y, cv=10, scoring='accuracy')
    #print(k, score.mean())

Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
1 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
2 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
3 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
4 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
5 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
6 ..............................................................
Empty DataFrame
Columns: [glm, tre

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
13 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
14 ..............................................................


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
15 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
16 ..............................................................
AUC_lr: 0.7410714285714286
(0.7333333333333333, 0.625, 0.8571428571428571)


...

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
27 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
28 ..............................................................
AUC_lr: 0.7410714285714286
(0.7333333333333333, 0.625, 0.8571428571428571)


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
29 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
30 ..............................................................
Empty DataFrame
Columns: [glm, tree, svm, RandomForest, knn, lda, qda, gnb, mbp, ada]
Index: []
31 ..............................................................


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [28]:
selectkBest=SelectKBest(f_classif,k=19)
 
    # 选择自变量
feature=train_X
 
    # 调用fit方法进行最好特征的选择
bestFeature=selectkBest.fit_transform(feature, train_Y)
 
    # 查看关键因子
    #print(selectkBest.get_support())
    #print(feature.columns[selectkBest.get_support()])
    #for i in range(30):
x = train_X[feature.columns[selectkBest.get_support()]]
#metric_all = pd.DataFrame()

from sklearn.linear_model import LogisticRegression
 
lr = LogisticRegression(C=1000, solver='liblinear', random_state=0)
pre_lr = lr.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_lr = roc_auc_score(val_Y, pre_lr)
#    if auc_lr>0.7:
print('AUC_lr:', auc_lr)
print(calculate_metric(val_Y, pre_lr))
    #metric.sort()
    #metric_all['glm'] = auc_lr
 
    # 拟合决策树模型
from sklearn import tree
 
tree = tree.DecisionTreeClassifier(criterion='gini')
pre_tree = tree.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_tree = roc_auc_score(val_Y, pre_tree)
    #if auc_tree>0.7:
print('AUC_tree:', auc_tree)
print(calculate_metric(val_Y, pre_tree))
    #metric = cross_val_score(tree, x, Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['tree'] = auc_tree
 
    # 拟合svm模型
from sklearn import svm
 
svc = svm.SVC(C=1.0, kernel='rbf', gamma='auto')
pre_svc = svc.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_svc = roc_auc_score(val_Y, pre_svc)
    #if auc_svc>0.7:
print('AUC_svc:', auc_svc)
print(calculate_metric(val_Y, pre_svc))
    #metric = cross_val_score(svc, x, Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['svm'] = auc_svc
 
    # 拟合随机森林算法
from sklearn.ensemble import RandomForestClassifier
 
RF = RandomForestClassifier(n_estimators=30, criterion='gini', random_state=10)
pre_RF = RF.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_RF = roc_auc_score(val_Y, pre_RF)
    #if auc_RF>0.7:
print('AUC_RF:', auc_RF)
print(calculate_metric(val_Y, pre_RF))
    #metric = cross_val_score(RF, x, Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['RandomForest'] = auc_RF
 
    # 构造knn最近邻模型
from sklearn import neighbors
 
knn = neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
pre_knn = knn.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_knn = roc_auc_score(val_Y, pre_knn)
    #if auc_knn>0.7:
print('AUC_knn:', auc_knn)
print(calculate_metric(val_Y, pre_knn))
    #metric = cross_val_score(estimator=knn, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['knn'] = auc_knn
 
    # 构造lda模型
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None, priors=None)
pre_lda = lda.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_lda = roc_auc_score(val_Y, pre_lda)
    #if auc_lda>0.7:
print('AUC_lda:', auc_lda)
print(calculate_metric(val_Y, pre_lda))
    #metric = cross_val_score(estimator=lda, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['lda'] = auc_lda
 
    # 构造qda模型
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
 
qda = QuadraticDiscriminantAnalysis()
pre_qda = qda.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_qda = roc_auc_score(val_Y, pre_qda)
    #if auc_qda>0.7:
print('AUC_qda:', auc_qda)
print(calculate_metric(val_Y, pre_qda))
    #metric = cross_val_score(estimator=qda, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['qda'] = auc_qda
 
    #  高斯朴素贝叶斯,多项式贝叶斯适用于文本分类，伯努利贝叶斯需要全部变量威二值变量
from sklearn.naive_bayes import GaussianNB
 
gnb = GaussianNB()
pre_gnb = gnb.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_gnb = roc_auc_score(val_Y, pre_gnb)
    #if auc_gnb>0.7:
print('AUC_gnb:', auc_gnb)
print(calculate_metric(val_Y, pre_gnb))
    #metric = cross_val_score(estimator=gnb, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['gnb'] = auc_gnb
 
    # sklearn提供了bp多层神经网络，隐含层设为 (4, 3, 2)
from sklearn.neural_network import MLPClassifier
 
mbp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 3, 2), random_state=1)
pre_mbp = mbp.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_mbp = roc_auc_score(val_Y, pre_mbp)
    #if auc_mbp>0.7:
print('AUC_mbp:', auc_mbp)
print(calculate_metric(val_Y, pre_mbp))
    #metric = cross_val_score(estimator=mbp, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['mbp'] = auc_mbp
 
    # 采用adaboost算法
from sklearn.ensemble import AdaBoostClassifier
 
    # 弱分类器的参数 base_estimator 默认为决策树
ada = AdaBoostClassifier(n_estimators=100)
pre_ada = ada.fit(x, train_Y).predict(val_X[feature.columns[selectkBest.get_support()]])
auc_ada = roc_auc_score(val_Y, pre_ada)
    #if auc_ada>0.7:
print('AUC_ada:', auc_ada)
print(calculate_metric(val_Y, pre_ada))
    #metric = cross_val_score(estimator=ada, X=x, y=Y, cv=10, scoring='accuracy')
    #metric.sort()
    #metric_all['ada'] = auc_ada
 
    # 将10种模型结果进行比较
    #print(metric_all)
print(k+1,"..............................................................")#,"个特征按照accuracy进行排序")
    #metric_mean = metric_all.mean()
    #print(metric_mean.sort_values(ascending=False))
    #score = cross_val_score(g, X[feature.columns[selectkBest.get_support()]], Y, cv=10, scoring='accuracy')
    #print(k, score.mean())

AUC_lr: 0.7410714285714286
(0.7333333333333333, 0.625, 0.8571428571428571)
AUC_tree: 0.5982142857142857
(0.6, 0.625, 0.5714285714285714)
AUC_svc: 0.4285714285714286
(0.4, 0.0, 0.8571428571428571)
AUC_RF: 0.5446428571428572
(0.5333333333333333, 0.375, 0.7142857142857143)
AUC_knn: 0.6160714285714286
(0.6, 0.375, 0.8571428571428571)
AUC_lda: 0.6696428571428572
(0.6666666666666666, 0.625, 0.7142857142857143)
AUC_qda: 0.5982142857142857
(0.6, 0.625, 0.5714285714285714)
AUC_gnb: 0.5089285714285715
(0.5333333333333333, 0.875, 0.14285714285714285)
AUC_mbp: 0.7410714285714286
(0.7333333333333333, 0.625, 0.8571428571428571)
AUC_ada: 0.5892857142857143
(0.6, 0.75, 0.42857142857142855)
31 ..............................................................


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [ ]:
X=df[df.columns[2:]]
Y=df['group']

In [41]:
from sklearn.model_selection import cross_val_score,cross_val_predict,  KFold

In [43]:
for k in range(31):
    selectkBest=SelectKBest(f_classif,k=k+1)
 
    # 选择自变量
 
    feature=X
 
    # 调用fit方法进行最好特征的选择
 
    bestFeature=selectkBest.fit_transform(feature, Y)
 
    # 查看关键因子
    #print(selectkBest.get_support())
    #print(feature.columns[selectkBest.get_support()])
    #for i in range(30):
    x = X[feature.columns[selectkBest.get_support()]]
    metric_all = pd.DataFrame()

    from sklearn.linear_model import LogisticRegression
 
    lr = LogisticRegression(C=1000, solver='liblinear', random_state=0)
    metric = cross_val_score(lr, X, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['glm'] = metric[::-1]
 
    # 拟合决策树模型
    from sklearn import tree
 
    tree = tree.DecisionTreeClassifier(criterion='gini')
    metric = cross_val_score(tree, X, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['tree'] = metric[::-1]

    # 拟合svm模型
    from sklearn import svm

    svc = svm.SVC(C=1.0, kernel='rbf', gamma='auto')
    metric = cross_val_score(svc, X, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['svm'] = metric[::-1]

    # 拟合随机森林算法
    from sklearn.ensemble import RandomForestClassifier

    RF = RandomForestClassifier(n_estimators=30, criterion='gini', random_state=10)
    metric = cross_val_score(RF, X, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['RandomForest'] = metric[::-1]

    # 构造knn最近邻模型
    from sklearn import neighbors

    knn = neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
    metric = cross_val_score(estimator=knn, X=X, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['knn'] = metric[::-1]

    # 构造lda模型
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

    lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None, priors=None)
    metric = cross_val_score(estimator=lda, X=X, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['lda'] = metric[::-1]

    # 构造qda模型
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

    qda = QuadraticDiscriminantAnalysis()
    metric = cross_val_score(estimator=qda, X=X, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['qda'] = metric[::-1]

    #  高斯朴素贝叶斯,多项式贝叶斯适用于文本分类，伯努利贝叶斯需要全部变量威二值变量
    from sklearn.naive_bayes import GaussianNB

    gnb = GaussianNB()
    metric = cross_val_score(estimator=gnb, X=X, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['gnb'] = metric[::-1]

    # sklearn提供了bp多层神经网络，隐含层设为 (4, 3, 2)
    from sklearn.neural_network import MLPClassifier

    mbp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 3, 2), random_state=1)
    metric = cross_val_score(estimator=mbp, X=X, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['mbp'] = metric[::-1]

    # 采用adaboost算法
    from sklearn.ensemble import AdaBoostClassifier

    # 弱分类器的参数 base_estimator 默认为决策树
    ada = AdaBoostClassifier(n_estimators=100)
    metric = cross_val_score(estimator=ada, X=X, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['ada'] = metric[::-1]

    # 将10种模型结果进行比较
    print(k+1,"个特征")
    print(metric_all)
    print("按照accuracy进行排序")
    metric_mean = metric_all.mean()
    print(metric_mean.sort_values(ascending=False))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

1 个特征
     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.750  1.000  0.750         0.875  0.875  0.875  0.625  0.750  0.5  0.750
1  0.625  0.750  0.750         0.750  0.750  0.750  0.625  0.750  0.5  0.750
2  0.625  0.750  0.625         0.750  0.750  0.750  0.500  0.625  0.5  0.625
3  0.625  0.750  0.625         0.625  0.750  0.625  0.500  0.625  0.5  0.625
4  0.625  0.750  0.500         0.625  0.750  0.625  0.400  0.500  0.5  0.500
5  0.500  0.625  0.500         0.625  0.625  0.625  0.375  0.500  0.5  0.500
6  0.500  0.500  0.500         0.600  0.625  0.600  0.250  0.500  0.5  0.500
7  0.500  0.500  0.500         0.500  0.500  0.375  0.250  0.500  0.5  0.500
8  0.375  0.500  0.375         0.500  0.300  0.375  0.250  0.500  0.5  0.375
9  0.375  0.500  0.300         0.375  0.250  0.375  0.125  0.500  0.5  0.250
按照accuracy进行排序
tree            0.6625
RandomForest    0.6225
knn             0.6175
lda             0.5975
gnb             0.5750
glm             0

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

2 个特征
     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.750  0.875  0.750         0.875  0.875  0.875  0.625  0.750  0.5  0.750
1  0.625  0.750  0.750         0.750  0.750  0.750  0.625  0.750  0.5  0.750
2  0.625  0.750  0.625         0.750  0.750  0.750  0.500  0.625  0.5  0.625
3  0.625  0.750  0.625         0.625  0.750  0.625  0.500  0.625  0.5  0.625
4  0.625  0.625  0.500         0.625  0.750  0.625  0.400  0.500  0.5  0.500
5  0.500  0.500  0.500         0.625  0.625  0.625  0.375  0.500  0.5  0.500
6  0.500  0.500  0.500         0.600  0.625  0.600  0.250  0.500  0.5  0.500
7  0.500  0.500  0.500         0.500  0.500  0.375  0.250  0.500  0.5  0.500
8  0.375  0.375  0.375         0.500  0.300  0.375  0.250  0.500  0.5  0.375
9  0.375  0.375  0.300         0.375  0.250  0.375  0.125  0.500  0.5  0.250
按照accuracy进行排序
RandomForest    0.6225
knn             0.6175
tree            0.6000
lda             0.5975
gnb             0.5750
glm             0

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

...

29 个特征
     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.750  0.875  0.750         0.875  0.875  0.875  0.625  0.750  0.5  0.750
1  0.625  0.875  0.750         0.750  0.750  0.750  0.625  0.750  0.5  0.750
2  0.625  0.750  0.625         0.750  0.750  0.750  0.500  0.625  0.5  0.625
3  0.625  0.750  0.625         0.625  0.750  0.625  0.500  0.625  0.5  0.625
4  0.625  0.625  0.500         0.625  0.750  0.625  0.400  0.500  0.5  0.500
5  0.500  0.625  0.500         0.625  0.625  0.625  0.375  0.500  0.5  0.500
6  0.500  0.500  0.500         0.600  0.625  0.600  0.250  0.500  0.5  0.500
7  0.500  0.500  0.500         0.500  0.500  0.375  0.250  0.500  0.5  0.500
8  0.375  0.500  0.375         0.500  0.300  0.375  0.250  0.500  0.5  0.375
9  0.375  0.375  0.300         0.375  0.250  0.375  0.125  0.500  0.5  0.250
按照accuracy进行排序
tree            0.6375
RandomForest    0.6225
knn             0.6175
lda             0.5975
gnb             0.5750
glm             

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

30 个特征
     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.750  0.875  0.750         0.875  0.875  0.875  0.625  0.750  0.5  0.750
1  0.625  0.875  0.750         0.750  0.750  0.750  0.625  0.750  0.5  0.750
2  0.625  0.750  0.625         0.750  0.750  0.750  0.500  0.625  0.5  0.625
3  0.625  0.750  0.625         0.625  0.750  0.625  0.500  0.625  0.5  0.625
4  0.625  0.750  0.500         0.625  0.750  0.625  0.400  0.500  0.5  0.500
5  0.500  0.750  0.500         0.625  0.625  0.625  0.375  0.500  0.5  0.500
6  0.500  0.625  0.500         0.600  0.625  0.600  0.250  0.500  0.5  0.500
7  0.500  0.500  0.500         0.500  0.500  0.375  0.250  0.500  0.5  0.500
8  0.375  0.500  0.375         0.500  0.300  0.375  0.250  0.500  0.5  0.375
9  0.375  0.500  0.300         0.375  0.250  0.375  0.125  0.500  0.5  0.250
按照accuracy进行排序
tree            0.6875
RandomForest    0.6225
knn             0.6175
lda             0.5975
gnb             0.5750
glm             

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

31 个特征
     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.750  0.875  0.750         0.875  0.875  0.875  0.625  0.750  0.5  0.750
1  0.625  0.750  0.750         0.750  0.750  0.750  0.625  0.750  0.5  0.750
2  0.625  0.625  0.625         0.750  0.750  0.750  0.500  0.625  0.5  0.625
3  0.625  0.625  0.625         0.625  0.750  0.625  0.500  0.625  0.5  0.625
4  0.625  0.625  0.500         0.625  0.750  0.625  0.400  0.500  0.5  0.500
5  0.500  0.500  0.500         0.625  0.625  0.625  0.375  0.500  0.5  0.500
6  0.500  0.500  0.500         0.600  0.625  0.600  0.250  0.500  0.5  0.500
7  0.500  0.500  0.500         0.500  0.500  0.375  0.250  0.500  0.5  0.500
8  0.375  0.500  0.375         0.500  0.300  0.375  0.250  0.500  0.5  0.375
9  0.375  0.500  0.300         0.375  0.250  0.375  0.125  0.500  0.5  0.250
按照accuracy进行排序
RandomForest    0.6225
knn             0.6175
tree            0.6000
lda             0.5975
gnb             0.5750
glm             